biased sampling scheme of:
- logit(sample_probability_centering) + \
                               0.1 * (data['x'].abs() - 25.5))

# Imports:

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

from scipy.stats import chi2

from tqdm import tqdm, trange
import random

In [2]:
import matplotlib.pyplot as plt
import pickle
import statsmodels.api as sm
from scipy.special import expit, logit

# Hyperparameters:

In [6]:
pop_size = 100 * 1_000
sample_size = 30
agg_iters = 100_0

beta = 1

## for biased sampling mechanism:
sample_probability_centering = 0.8
sample_probability_bias_factor = 0.1

In [7]:
## for rng generator:
pop_number = 1

# Run:

In [8]:
rand_generator = np.random.default_rng(seed=333 * pop_number)

In [9]:
assert pop_size % 100 == 0
base_x = [i for i in range(-50, 0)] + [i for i in range(1, 51)]
x = pd.Series(base_x * (pop_size // 100))
normal_stdev = 30

In [11]:
fitted_betas = []
fitted_FI = []
fitted_correlations = []
realized_sample_sizes = []

Create the simulated data (x, y):

In [12]:
y = x * beta + rand_generator.normal(size=pop_size) * normal_stdev

data = pd.concat([y, x], axis=1)

data.columns = ['y', 'x']

fitted_beta = sm.OLS(data['y'], data['x']).fit().params.iloc[0]

data['g_star'] = data['x'] * (data['y'] - data['x'] * fitted_beta)

In [13]:
marginal_probabilities = expit(logit(sample_probability_centering) + \
                               sample_probability_bias_factor * (data['x'].abs() - 25.5))

# run:

In [14]:
for _ in tqdm(range(agg_iters)):
    srs_sample_indices = pd.Series(random.sample(range(pop_size), sample_size))
    srs_sample_indices = srs_sample_indices.sort_values().reset_index(drop=True)

    # reset the sample
    data['r0'] = 0
    data['r'] = 0
    data['r1'] = 0
    data['r2'] = 0
    
    data.loc[srs_sample_indices, 'r0'] = 1
    full_sampled_data = data[data['r0'] == 1]

    marginal_probabilities = expit(logit(sample_probability_centering) + \
                                   sample_probability_bias_factor * (full_sampled_data['x'].abs() - 25.5))
    biased_sample_indices = marginal_probabilities.index[rand_generator.binomial(n=1, p = marginal_probabilities) == 1]

    data.loc[biased_sample_indices, 'r'] = 1

    biased_sample_indices_size = len(biased_sample_indices)
    realized_sample_sizes.append(biased_sample_indices_size)


    srs_sample = data[data['r0'] == 1].copy()
    biased_sample = data[data['r'] == 1].copy()

    srs_fitted_beta = sm.OLS(srs_sample['y'], srs_sample['x']).fit().params.iloc[0]
    biased_fitted_beta = sm.OLS(biased_sample['y'], biased_sample['x']).fit().params.iloc[0]

    fitted_betas.append(pd.Series([fitted_beta, srs_fitted_beta, biased_fitted_beta]))

    fitted_FI.append(pd.Series([np.mean(srs_sample['x'] ** 2), np.mean(biased_sample['x'] ** 2)]))

    fitted_correlations.append(data[['g_star', 'r0', 'r']].corr().iloc[0, 1:])

100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:04<00:00, 203.63it/s]


Clean up the data:

In [17]:
fitted_betas = pd.concat(fitted_betas, axis=1).T
fitted_FI = pd.concat(fitted_FI, axis=1).T
fitted_correlations = pd.concat(fitted_correlations, axis=1).T.reset_index(drop=True)

In [18]:
fitted_FI.columns = ['srs', 'biased']
fitted_betas.columns = ['population', 'srs', 'biased']
fitted_correlations.columns = ['srs', 'biased']

In [19]:
realized_sample_sizes = pd.Series(realized_sample_sizes)

In [20]:
fitted_FI.head()

,srs,biased
0,1069.866667,1182.407407
1,1007.600000,1272.391304
2,959.100000,1390.000000
3,927.733333,1373.722222
4,949.000000,1166.916667


In [21]:
fitted_FI.tail()

,srs,biased
995,1231.133333,1313.560000
996,945.166667,1226.818182
997,863.400000,1192.210526
998,1100.533333,1308.826087
999,871.700000,1093.045455


In [22]:
fitted_FI_and_corr = fitted_FI.join(fitted_correlations, lsuffix = '_fi', rsuffix = '_corr')

fitted_FI_and_corr.head()

,srs_fi,biased_fi,srs_corr,biased_corr
0,1069.866667,1182.407407,0.002088,0.002220
1,1007.600000,1272.391304,-0.001691,-0.000874
2,959.100000,1390.000000,0.002868,0.002634
3,927.733333,1373.722222,-0.001650,-0.003755
4,949.000000,1166.916667,-0.009547,-0.011470


In [18]:
np.mean(data['x'] ** 2)

np.float64(858.5)

# Save:

In [23]:
Jeee = np.mean(data['x'] ** 2)

fitted_FI_and_corr['srs_samp_elasticity'] = fitted_FI_and_corr.eval(f'srs_fi / {Jeee}')
fitted_FI_and_corr['biased_samp_elasticity'] = fitted_FI_and_corr.eval(f'biased_fi / {Jeee}')

In [24]:
fitted_FI_and_corr.describe()

,srs_fi,biased_fi,srs_corr,biased_corr,srs_samp_elasticity,biased_samp_elasticity
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
mean,864.773433,1076.788951,-0.000189,-0.000187,1.007307,1.254268
std,143.207731,167.695222,0.003115,0.003508,0.166812,0.195335
min,463.966667,589.866667,-0.009918,-0.011470,0.540439,0.687090
25%,760.441667,952.637124,-0.002160,-0.002533,0.885779,1.109653
50%,863.600000,1074.710714,-0.000060,-0.000217,1.005941,1.251847
75%,962.083333,1192.388834,0.001999,0.002397,1.120656,1.388921
max,1327.333333,1553.480000,0.008405,0.009456,1.546108,1.809528


In [ ]:
fitted_FI_and_corr.to_pickle('sample_by_x_bias.pkl')